In [26]:
# Módulos necesarios
!pip install scikit-learn imblearn
!pip install xgboost
!pip install lightgbm
!pip install catboost

import numpy as np
import pandas as pd
import matplotlib as plt
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from scipy.stats import expon, reciprocal
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Clasificadores
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier, AdaBoostClassifier, BaggingClassifier, VotingClassifier, StackingClassifier, HistGradientBoostingClassifier
# from sklearn.svm import SVC
# from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import RidgeClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

In [27]:
data = pd.read_csv('C:\\Users\\josan\\Documents\\GitHub\\EDEM_MDA2324\\Alumnos\\ES\\Josan_Rodrigo_Cortes\\Dataproject3\\train.csv') # set de entrenamiento
test = pd.read_csv('C:\\Users\\josan\\Documents\\GitHub\\EDEM_MDA2324\\Alumnos\\ES\\Josan_Rodrigo_Cortes\Dataproject3\\test.csv') # set de test

In [28]:
# Quitamos los duplicados y comprobamos
data = data.drop_duplicates(keep = False)
test = test.drop_duplicates(keep = False)

In [29]:
# Reemplazamos los valores no imputados ('?', valores no posibles donde corresponda)

# Reemplazamos '?' por NaN en todos los dataset

data.replace('?', np.nan, inplace=True)
test.replace('?', np.nan, inplace=True)

# Reemplazamos '0' por NaN en las columnas 'trestbps', 'chol', 'slope' y 'thal'

# Especificamos las columnas donde reemplazar
columns_to_replace = ['trestbps', 'chol', 'slope', 'thal']
all_columns = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']

# Reemplazamos '0', '0.0' por NaN
for column in columns_to_replace:
    # Set de entrenamiento
    data[column] = data[column].replace([0, '0', 0.0, '0.0'], np.nan)
    
    # Set de test
    test[column] = test[column].replace([0, '0', 0.0, '0.0'], np.nan)

# Reemplazamos todos los valores negativos en los dataset por NaN
for column in all_columns:
    # Set de entrenamiento
    data[column] = data[column].astype(float)
    data[column] = data[column].apply(lambda x: np.nan if x < 0 else x)
    # Set de test
    test[column] = test[column].astype(float)
    test[column] = test[column].apply(lambda x: np.nan if x < 0 else x)

# Finalmente, llenamos todas las entradas vacías con NaN

# Set de entrenamiento
data.replace(" ", np.nan, inplace=True)
data.replace("", np.nan, inplace=True)
# Set de test
test.replace(" ", np.nan, inplace=True)
test.replace("", np.nan, inplace=True)

In [30]:
# Valor suelto en set de test, columna 'ca'
test['ca'] = test['ca'].replace(9.0, 0.0)

In [31]:
# Ahora, probamos estrategias para inputar valores en los NaNs

# Inicializamos SimpleImputer
imputer = SimpleImputer(strategy='mean')

columns_imputer = ['trestbps', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak']
columns_ml = ['chol', 'slope', 'ca', 'thal']

# Hacemos fit a SimpleImputer y computamos
data[columns_imputer] = imputer.fit_transform(data[columns_imputer])
test[columns_imputer] = imputer.transform(test[columns_imputer])

In [32]:
# Ahora, aplicamos un modelo ML (Iterative Imputer) para las columnas más 'dañadas'

# Initialize the Iterative Imputer
iterative_imputer = IterativeImputer()

# Apply the imputer to the dataset
data[columns_ml] = iterative_imputer.fit_transform(data[columns_ml])
test[columns_ml] = iterative_imputer.transform(test[columns_ml])



In [33]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 730 entries, 0 to 731
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       730 non-null    float64
 1   sex       730 non-null    float64
 2   cp        730 non-null    float64
 3   trestbps  730 non-null    float64
 4   chol      730 non-null    float64
 5   fbs       730 non-null    float64
 6   restecg   730 non-null    float64
 7   thalach   730 non-null    float64
 8   exang     730 non-null    float64
 9   oldpeak   730 non-null    float64
 10  slope     730 non-null    float64
 11  ca        730 non-null    float64
 12  thal      730 non-null    float64
 13  label     730 non-null    int64  
dtypes: float64(13), int64(1)
memory usage: 85.5 KB


In [34]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 184 entries, 0 to 183
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       184 non-null    float64
 1   sex       184 non-null    float64
 2   cp        184 non-null    float64
 3   trestbps  184 non-null    float64
 4   chol      184 non-null    float64
 5   fbs       184 non-null    float64
 6   restecg   184 non-null    float64
 7   thalach   184 non-null    float64
 8   exang     184 non-null    float64
 9   oldpeak   184 non-null    float64
 10  slope     184 non-null    float64
 11  ca        184 non-null    float64
 12  thal      184 non-null    float64
dtypes: float64(13)
memory usage: 20.1 KB


In [44]:
# En este paso, normalizamos  y estandarizamos los datos según su distribución

cols_gaussian = ['age', 'trestbps', 'chol', 'thalach']
cols_norm = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'oldpeak', 'slope', 'ca', 'thal']

scaler_std = StandardScaler()
data[cols_gaussian] = scaler_std.fit_transform(data[cols_gaussian])
test[cols_gaussian] = scaler_std.transform(test[cols_gaussian])

scaler_norm = MinMaxScaler(feature_range=(0, 1))
data[cols_norm] = scaler_norm.fit_transform(data[cols_norm])
test[cols_norm] = scaler_norm.transform(test[cols_norm])

In [45]:
# Dividimos data en sets de predictores y targets
X = data.drop('label', axis=1)  # Features (variables independientes)
y = data['label']  # Variable objetivo (variable dependiente)

In [52]:
y.unique()

array([0, 2, 3, 4, 1], dtype=int64)

In [46]:
data['label'] = data['label'].round(0)

In [47]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15, random_state=42)
X_test = test

In [53]:

modelos_mapeados = {
    "Regresión Logística": LogisticRegression,
    "K-Nearest Neighbors (KNN)": KNeighborsClassifier,
    "Árboles de Decisión": DecisionTreeClassifier,
    "Random Forest": RandomForestClassifier,
    "Gradient Boosting": GradientBoostingClassifier,
    "Máquinas de Vectores de Soporte (SVM)": SVC,
    "Redes Neuronales": MLPClassifier,
    "AdaBoost": AdaBoostClassifier,
    "XGBoost": XGBClassifier,
    "LDA (Análisis Discriminante Lineal)": LinearDiscriminantAnalysis,
    "QDA (Análisis Discriminante Cuadrático)": QuadraticDiscriminantAnalysis,
    "Clasificador de Vecinos Ponderados (WKNN)": KNeighborsClassifier,
    "Extra Trees": ExtraTreesClassifier,
    "Perceptrón Multicapa (MLP)": MLPClassifier,
    "LightGBM": LGBMClassifier,
    "CatBoost": CatBoostClassifier,
    # "Voting Classifier": VotingClassifier,
    # "Stacking Classifier": StackingClassifier,
    "Bagging Classifier": BaggingClassifier,
    "Histogram-Based Gradient Boosting": HistGradientBoostingClassifier,
    "Linear Discriminant Analysis (LDA)": LinearDiscriminantAnalysis,
    "Quadratic Discriminant Analysis (QDA)": QuadraticDiscriminantAnalysis,
    "Gaussian Process Classifier": GaussianProcessClassifier,
    "Decision Tree Classifier": DecisionTreeClassifier,
    "Gaussian Naive Bayes": GaussianNB,
    "Calibrated Classifier": CalibratedClassifierCV,
    "Ridge Classifier": RidgeClassifier,
    "Support Vector Classifier (SVC)": SVC
}

# Ahora tienes un diccionario donde las claves son los nombres de los modelos y los valores son las clases de modelos correspondientes
# Por ejemplo, para obtener la clase de modelo para "Regresión Logística":
# modelo_regresion_logistica = modelos_mapeados["Regresión Logística"]()



In [54]:
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

# Crear una lista para almacenar la precisión de cada modelo
precisiones = []

# Iterar sobre los modelos de clasificación
for modelo_nombre in modelos_mapeados.keys():
    # Crear un nuevo modelo para cada iteración
    modelo_clase = modelos_mapeados[modelo_nombre]  # Obtener la clase del modelo
    modelo = modelo_clase()  # Crear una instancia del modelo
    
    # Entrenar el clasificador con los datos de entrenamiento
    modelo.fit(X_train, y_train)
    
    # Hacer predicciones en los datos de validación
    y_pred_val = modelo.predict(X_val)
    
    # Calcular la precisión del modelo en los datos de validación
    precision = accuracy_score(y_val, y_pred_val)
    
    # Calcular el puntaje F1 del modelo en los datos de validación
    puntaje_f1 = f1_score(y_val, y_pred_val, average='weighted')  # 'macro' es útil cuando tienes clases desbalanceadas
    
    # Almacenar la precisión y el puntaje F1 del modelo
    precisiones.append((modelo_nombre, precision, puntaje_f1))

    print(f"Precisión del modelo {modelo_nombre}:", precision)
    print(f"Puntaje F1 del modelo {modelo_nombre}:", puntaje_f1)
    
    # Calcular y mostrar la matriz de confusión
    matriz_confusion = confusion_matrix(y_val, y_pred_val)
    print(f"Matriz de confusión para el modelo {modelo_nombre}:")
    print(matriz_confusion)
    print("\n")

Precisión del modelo Regresión Logística: 0.6363636363636364
Puntaje F1 del modelo Regresión Logística: 0.624910644910645
Matriz de confusión para el modelo Regresión Logística:
[[50  6  0  1  0]
 [ 6  9  1  6  0]
 [ 1  3  4  2  0]
 [ 3  4  1  7  1]
 [ 0  1  0  4  0]]


Precisión del modelo K-Nearest Neighbors (KNN): 0.5363636363636364
Puntaje F1 del modelo K-Nearest Neighbors (KNN): 0.5181671435908723
Matriz de confusión para el modelo K-Nearest Neighbors (KNN):
[[45  7  4  1  0]
 [ 8  8  3  3  0]
 [ 3  2  4  0  1]
 [ 5  3  6  2  0]
 [ 0  2  1  2  0]]


Precisión del modelo Árboles de Decisión: 0.4818181818181818
Puntaje F1 del modelo Árboles de Decisión: 0.5145495194680758
Matriz de confusión para el modelo Árboles de Decisión:
[[39 10  4  3  1]
 [ 2  9  4  5  2]
 [ 0  1  4  3  2]
 [ 3  2  8  1  2]
 [ 0  0  1  4  0]]


Precisión del modelo Random Forest: 0.5818181818181818
Puntaje F1 del modelo Random Forest: 0.577005217005217
Matriz de confusión para el modelo Random Forest:
[[48  6

C:\Users\josan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\josan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Precisión del modelo Redes Neuronales: 0.6
Puntaje F1 del modelo Redes Neuronales: 0.5910610810297331
Matriz de confusión para el modelo Redes Neuronales:
[[50  3  4  0  0]
 [ 6  7  3  6  0]
 [ 0  0  6  4  0]
 [ 3  1  9  3  0]
 [ 0  0  1  4  0]]


Precisión del modelo AdaBoost: 0.5272727272727272
Puntaje F1 del modelo AdaBoost: 0.5314223575467595
Matriz de confusión para el modelo AdaBoost:
[[46  5  3  2  1]
 [ 6  6  4  5  1]
 [ 1  1  2  5  1]
 [ 3  4  4  3  2]
 [ 1  0  2  1  1]]


Precisión del modelo XGBoost: 0.5636363636363636
Puntaje F1 del modelo XGBoost: 0.5721471628729693
Matriz de confusión para el modelo XGBoost:
[[45  2  6  4  0]
 [ 6  8  3  3  2]
 [ 1  1  5  2  1]
 [ 3  2  5  4  2]
 [ 0  1  2  2  0]]


Precisión del modelo LDA (Análisis Discriminante Lineal): 0.6
Puntaje F1 del modelo LDA (Análisis Discriminante Lineal): 0.5992683828449833
Matriz de confusión para el modelo LDA (Análisis Discriminante Lineal):
[[48  8  0  1  0]
 [ 4 10  2  5  1]
 [ 1  3  2  3  1]
 [ 3  5  1 

C:\Users\josan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Precisión del modelo Perceptrón Multicapa (MLP): 0.5909090909090909
Puntaje F1 del modelo Perceptrón Multicapa (MLP): 0.5897728853933104
Matriz de confusión para el modelo Perceptrón Multicapa (MLP):
[[49  5  2  1  0]
 [ 5  8  5  4  0]
 [ 1  0  4  5  0]
 [ 3  1  7  4  1]
 [ 0  0  1  4  0]]


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000084 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 584
[LightGBM] [Info] Number of data points in the train set: 620, number of used features: 13
[LightGBM] [Info] Start training from score -0.838732
[LightGBM] [Info] Start training from score -1.531880
[LightGBM] [Info] Start training from score -1.844752
[LightGBM] [Info] Start training from score -1.918860
[LightGBM] [Info] Start training from score -3.062424
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [

In [50]:
# Modelo elegido:

modelo = LogisticRegression()  # Crear una instancia del modelo

# Entrenar el clasificador con los datos de entrenamiento
modelo.fit(X_train, y_train)

# Hacer predicciones en los datos de validación
y_pred_val = modelo.predict(X_val)

# Calcular la precisión del modelo en los datos de validación
precision = accuracy_score(y_val, y_pred_val)

# Calcular el puntaje F1 del modelo en los datos de validación
puntaje_f1 = f1_score(y_val, y_pred_val, average='weighted')  # 'macro' es útil cuando tienes clases desbalanceadas

# Almacenar la precisión y el puntaje F1 del modelo
precisiones.append((modelo_nombre, precision, puntaje_f1))

print(f"Precisión del modelo {modelo}:", precision)
print(f"Puntaje F1 del modelo {modelo}:", puntaje_f1)

# Calcular y mostrar la matriz de confusión
matriz_confusion = confusion_matrix(y_val, y_pred_val)
print(f"Matriz de confusión para el modelo {modelo_nombre}:")
print(matriz_confusion)
print("\n")

Precisión del modelo LogisticRegression(): 0.6363636363636364
Puntaje F1 del modelo LogisticRegression(): 0.624910644910645
Matriz de confusión para el modelo Support Vector Classifier (SVC):
[[50  6  0  1  0]
 [ 6  9  1  6  0]
 [ 1  3  4  2  0]
 [ 3  4  1  7  1]
 [ 0  1  0  4  0]]




## Voting, no mejora el F1 score anterior

In [61]:

from sklearn.datasets import load_iris

# Cargar datos
iris = load_iris()
X, y = iris.data, iris.target


# Definir los clasificadores individuales
clf1 = LogisticRegression(max_iter=1000)
clf2 = GaussianNB()
clf3 = CatBoostClassifier(verbose=0)
clf4 = RandomForestClassifier()

# Crear el Voting Classifier
voting_clf = VotingClassifier(estimators=[
    ('LogReg', clf1),
    ('GausNB', clf2),
    ('CatBoot', clf3),
    ('RandFor', clf4),
    
], voting='soft')

# Entrenar el Voting Classifier
voting_clf.fit(X_train, y_train)

# Evaluar el modelo
y_pred = voting_clf.predict(X_test)

# Calcular la precisión del modelo en los datos de validación
precision = accuracy_score(y_val, y_pred_val)

# Calcular el puntaje F1 del modelo en los datos de validación
puntaje_f1 = f1_score(y_val, y_pred_val, average='weighted')  # 'macro' es útil cuando tienes clases desbalanceadas

# Almacenar la precisión y el puntaje F1 del modelo
precisiones.append(('Voting Clasifier', precision, puntaje_f1))

print(f"Precisión del modelo Voting Clasifier:", precision)
print(f"Puntaje F1 del modelo Voting Clasifier:", puntaje_f1)

# Calcular y mostrar la matriz de confusión
matriz_confusion = confusion_matrix(y_val, y_pred_val)
print(f"Matriz de confusión para el modelo Voting Clasifier:")
print(matriz_confusion)
print("\n")


Precisión del modelo Voting Clasifier: 0.5909090909090909
Puntaje F1 del modelo Voting Clasifier: 0.5576797857789593
Matriz de confusión para el modelo Voting Clasifier:
[[49  5  2  1  0]
 [ 7 10  2  3  0]
 [ 2  2  5  1  0]
 [ 6  2  7  1  0]
 [ 0  3  0  2  0]]




El modelo Stacking_clf tampoco nos ofrece mejores resultado.


In [62]:

# Definir los clasificadores base
base_learners = [
    ('lr', LogisticRegression(max_iter=1000)),
    ('gnb', GaussianNB()),
    ('catboost', CatBoostClassifier(verbose=0)),
    ('rf', RandomForestClassifier())
]

# Clasificador final
final_estimator = LogisticRegression()

# Crear el Stacking Classifier
stacking_clf = StackingClassifier(
    estimators=base_learners,
    final_estimator=final_estimator,
    cv=5  # Número de pliegues en la validación cruzada
)
# Entrenar el Voting Classifier
stacking_clf.fit(X_train, y_train)

# Evaluar el modelo
y_pred = stacking_clf.predict(X_test)

# Calcular la precisión del modelo en los datos de validación
precision = accuracy_score(y_val, y_pred_val)

# Calcular el puntaje F1 del modelo en los datos de validación
puntaje_f1 = f1_score(y_val, y_pred_val, average='weighted')  # 'macro' es útil cuando tienes clases desbalanceadas

# Almacenar la precisión y el puntaje F1 del modelo
precisiones.append(('Voting Clasifier', precision, puntaje_f1))

print(f"Precisión del modelo stacking_clf:", precision)
print(f"Puntaje F1 del modelo stacking_clf:", puntaje_f1)

# Calcular y mostrar la matriz de confusión
matriz_confusion = confusion_matrix(y_val, y_pred_val)
print(f"Matriz de confusión para el modelo stacking_clf:")
print(matriz_confusion)
print("\n")


Precisión del modelo stacking_clf: 0.5909090909090909
Puntaje F1 del modelo stacking_clf: 0.5576797857789593
Matriz de confusión para el modelo stacking_clf:
[[49  5  2  1  0]
 [ 7 10  2  3  0]
 [ 2  2  5  1  0]
 [ 6  2  7  1  0]
 [ 0  3  0  2  0]]




In [51]:
# Montamos el DF para el generar el archivo csv para Kaggle
# Suponiendo que X_test es tu conjunto de datos de prueba
X_test = test
# Generar predicciones en el conjunto de datos de prueba
predicciones = modelo.predict(X_test)

# Crear DataFrame de predicciones
prediccion = pd.DataFrame({'ID': range(len(test)), 'label': predicciones})

# Guardar las predicciones en un archivo CSV
prediccion.to_csv('110424JosanCuadernoBaseRegresiónLogística.csv', index=False)